In [1]:
import numpy as np
import networkx as nx
import csv
import time
from numpy import linalg as LA
import itertools
import multiprocessing as mp
import pandas as pd
import concurrent.futures
import time

In [2]:
import subprocess
import os

cmd = subprocess.Popen('pwd', stdout=subprocess.PIPE)
cmd_out, cmd_err = cmd.communicate()
local_path = os.fsdecode(cmd_out).strip()

In [12]:
import os

directory_in_str = local_path

directory = os.fsencode(local_path+'/data/box_graph_data_2018_100/')

files = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.startswith('network_data_box_sampling_nooverlaps_m_100_'):
        files.append(filename)


In [ ]:
# from scipy.stats import hmean 

# def eff_radius(weights,combination):
#     resistances = [l/r**2 for r,l in weights]
#     if combination == 'parallel':
#         eff_resistance = hmean(resistances)/len(resistances)
#         eff_length = np.mean([l for r,l in weights])
#     else:
#         eff_resistance = sum(resistances)
#         eff_length = sum([l for r,l in weights])
#     return math.sqrt(eff_length/eff_resistance)

In [4]:
"""
Returns a graph constructed from edge connections data stored in data
"""

def graph(data):
    G = nx.Graph()
    for edge in data:
        G.add_edge(int(edge[0]), int(edge[1]))
        
    return G

In [2]:
"""
Input: g is a networkx graph object.

returns a minimal cycle basis of g. 
"""

def cycle_basis(g):
    return nx.minimum_cycle_basis(g)

In [3]:
"""
Inputs: min_basis is a list, H is a networkx graph object.

returns edges of the cycles in the basis and a grouped list of edge indices, where the
group index is the cycle index in the cycle basis list. 

This extra step is required because the minimal basis computed by NetworkX does not 
necessarily return the nodesin each cycle in the order in which they're connected, 
meaning one cannot assume that the consecutive nodes form edges in the network graph.

"""

def after_cycle_basis(min_basis,H):
    ordered_cycles = []
    i = 0
    j = -1
    for f in min_basis:
        ordered_cycles.append(list(nx.all_simple_paths(H, source=f[0], target=f[len(f)-1], cutoff = len(f)-1)))
        j = j + len(f)
        i = i + 1

    i = 0
    cycle_basis_edges = []
    for paths in ordered_cycles:
        first_route = True
        e1=[]
        e2=[]
        for path in paths:
            if set(min_basis[i]) == set(path):
                e1 = list(zip(path, path[1:] + path[:1]))
            else:
                if set(path) < set(min_basis[i]):
                    if first_route:
                        e2=list(zip(path[:-1],path[1:]))
                        first_route=False
                    else:
                        l=list(zip(path[1:],path[:-1]))
                        l.reverse()
                        e2=e2+l

        if e1:
            cycle_basis_edges.append(e1)
        else:
            cycle_basis_edges.append(e2)
                
        i = i + 1
        
    return cycle_basis_edges

In [ ]:
# def compute_basis_loops(n):
#     G = graph(graph_data[markers[n]+2:markers[n+1]]) #construct the network graph
    
#     basis = cycle_basis(G)  #compute minimal basis
    
#     cycle_basis_edges = after_cycle_basis(basis,G) #compute the edges in the basis cycles
#     return [len(basis), markers[n], cycle_basis_edges]

In [4]:
## main function to identify basis cycles/loops that are possibly entangled

def entanglement(n):
    G = graph(graph_data[markers[n]+2:markers[n+1]]) #construct the network graph
    
    basis = cycle_basis(G)  #compute minimal basis
    
    if len(basis)>1:    
        cycle_basis_edges = after_cycle_basis(basis,G) #compute the edges in the basis cycles
        cycle_basis_nodes = [np.array(edges).flatten() for edges in cycle_basis_edges] #for the sake of speeding up

        non_overlapping_basis_loops = {}
        for i in range(len(cycle_basis_nodes)):
            for cycle in cycle_basis_nodes[i+1:]:
                if len(np.intersect1d(cycle_basis_nodes[i],cycle)) == 0:
                    if i not in non_overlapping_basis_loops:
                        non_overlapping_basis_loops[i] = [cycle.tolist()]
                    else:
                        if cycle.tolist() not in non_overlapping_basis_loops[i]:
                            non_overlapping_basis_loops[i] = non_overlapping_basis_loops[i] + [cycle.tolist()]
                        else:
                            pass

        proximal_basis_loops = {}  #second filter
        for key in non_overlapping_basis_loops:
            c1 = np.mean(np.array(df_coordinates.loc[basis[key]]),axis=0)
            r1 = max([LA.norm(c1 - df_coordinates.loc[v]) for v in basis[key]])
            for cycle in non_overlapping_basis_loops[key]:
                c2 = np.mean(np.array(df_coordinates.loc[np.unique(cycle)]),axis=0)
                r2 = max([LA.norm(c2 - df_coordinates.loc[v]) for v in np.unique(cycle)])
                if (LA.norm(c1-c2) <= r1 + r2):
                    if (key not in proximal_basis_loops):
                        proximal_basis_loops[key] = [cycle]
                    else:
                        proximal_basis_loops[key] = proximal_basis_loops[key] + [cycle]



        possibly_entangled_loops = [] #final/third filter
        for key,value in proximal_basis_loops.items():
            linking_number = 0
            c = np.mean(np.array(df_coordinates.loc[basis[key]]),axis=0)
            for cycle in value:
                for k in range(0,len(cycle),2):
                    r2 = np.array(df_coordinates.loc[cycle[k]])
                    r1 = np.array(df_coordinates.loc[cycle[k+1]])
                    for tup in cycle_basis_edges[key]:
                        u = np.array(df_coordinates.loc[tup[0]]) - c
                        v = np.array(df_coordinates.loc[tup[1]]) - c
                        w = np.cross(u,v)
                        what = w/LA.norm(w)
                        s = np.dot(what,r2-r1)
                        if s != 0.0:
                            t = np.dot(what,c-r1)/s
                            if abs(t) <= 1.0:
                            #if t >= 0.0 and t <= 1.0:
                                r=t*r2+(1-t)*r1
                                if LA.norm(w)/2 < (LA.norm(np.cross(r-c,u))/2) + (LA.norm(np.cross(r-c,v))/2) +\
                                (LA.norm(np.cross(r-df_coordinates.loc[tup[0]],r-df_coordinates.loc[tup[1]]))/2):
                                    pass
                                else:
                                    linking_number = linking_number + np.sign(s)

                if linking_number != 0:
                    possibly_entangled_loops.append([linking_number, cycle, cycle_basis_nodes[key].tolist()])

        return [len(basis), markers[n], possibly_entangled_loops]

In [8]:
global graph_data
global markers
global coordinates

coordinates = []

with open(local_path + "/coordinates_2018.csv") as csvfile:
    filereader = csv.reader(csvfile)
    for row in filereader:
        coordinates.append([float(item) for item in row])
        
df_coordinates = pd.DataFrame(data=coordinates,columns=['V','VX','VY','VZ'])
df_coordinates['index'] = [int(x) for x in df_coordinates['V']]
df_coordinates = df_coordinates.drop(columns = ['V'])
df_coordinates = df_coordinates.set_index('index')

In [10]:
df_coordinates.head(5)

,VX,VY,VZ
index,,,
1,3030.000000,3891.000000,4.0
2,2463.000000,3822.000000,6.0
14,2590.000000,3882.000000,7.0
3,2605.000000,3895.000000,6.0
4,2107.333333,4131.333333,6.0


#### There are multiedges i.e. more than one edge between a pair of vertices. However, for the sake of what we're doing it serves the purpose to ignore that aspect and pick one edge among the collection. 

In [1]:
for file in files:
    print(file)
    graph_data = []
    with open(local_path + "/data/box_graph_data_2018_100/"+file,encoding = 'utf-8') as csvfile:
        filereader = csv.reader(csvfile)
        for row in filereader:
            graph_data.append([float(item) for item in row])
            
    markers = []
    for k in range(len(graph_data)):
        if graph_data[k] == [0,0,0,0,0,0]:
            markers.append(k)
        
    markers.append(len(graph_data))
    
    with concurrent.futures.ProcessPoolExecutor() as executor:
        results = executor.map(entanglement,range(len(markers)-1))
        
        for result in results:
            if result!=None:
                if result[2]:
                    filename = 'linked_loops_m_100_' + file.split('.')[0].split("_")[-1] + '.csv'
                    print(filename)
                    with open(local_path +'/data/entanglement_data/entanglement_data_2018/'+ filename, 'w+') as f:
                        filewriter = csv.writer(f)
                        filewriter.writerow([result[0],result[1]])
                        for lis in result[2]:
                            filewriter.writerow([lis[0]])
                            filewriter.writerow(lis[1])
                            filewriter.writerow(lis[2])
                        

#### Some analysis to check if the code is working as intended

In [ ]:
# from numpy import linalg as LA

# def linking(cycle1,cycle2):
#     linking_number=0
#     c=np.mean(cycle2,axis=0)
#     for k in range(len(cycle1)-1):
#         r2 = cycle1[k]
#         r1 = cycle1[k+1]
#         for j in range(len(cycle2)-1):
#             u = cycle2[j] - c
#             v = cycle2[j+1] - c
#             w = np.cross(u,v)
#             what=w/LA.norm(w)
#             s = np.dot(what,r2-r1)
#             if s != 0.0:
#                 t = np.dot(what,c-r1)/s
#                 #if t >= 0.0 and t <= 1.0:
#                 if abs(t) <= 1.0:
#                     r=t*r2+(1-t)*r1
#                     if LA.norm(w)/2 < (LA.norm(np.cross(r-c,u))/2) + (LA.norm(np.cross(r-c,v))/2) +\
#                     (LA.norm(np.cross(r-cycle2[j],r-cycle2[j+1]))/2):
#                         pass
#                     else:
#                         print(r)
#                         linking_number = linking_number + np.sign(s)
                    
#     return linking_number

In [ ]:
# def area(p1,p2,p3):
#     return ((p1[0]-p3[0])*(p2[1]-p3[1])-(p2[0] - p3[0])*(p1[1] - p3[1]))/2

In [ ]:
# def point_in_triangle(p,v1,v2,v3):
#      if abs(area(p,v1,v2))+abs(area(p,v2,v3))+abs(area(p,v3,v1))>abs(area(v1,v2,v3)):
#         return False
#      else:
#         return True